In [1]:
%%bash
cat << __JSON__ > zero_infer.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_16bit_weights_on_model_save": true
    },
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
__JSON__

In [2]:
%%bash
ls *.json

zero_infer.json


In [3]:
%%bash
cat -n zero_infer.json

     1	{
     2	    "fp16": {
     3	        "enabled": "auto",
     4	        "loss_scale": 0,
     5	        "loss_scale_window": 1000,
     6	        "initial_scale_power": 16,
     7	        "hysteresis": 2,
     8	        "min_loss_scale": 1
     9	    },
    10	    "zero_optimization": {
    11	        "stage": 3,
    12	        "offload_optimizer": {
    13	            "device": "cpu",
    14	            "pin_memory": true
    15	        },
    16	        "offload_param": {
    17	            "device": "cpu",
    18	            "pin_memory": true
    19	        },
    20	        "overlap_comm": true,
    21	        "contiguous_gradients": true,
    22	        "sub_group_size": 1e9,
    23	        "reduce_bucket_size": "auto",
    24	        "stage3_prefetch_bucket_size": "auto",
    25	        "stage3_param_persistence_threshold": "auto",
    26	        "stage3_max_live_parameters": 1e9,
    27	        "stage3_max_reuse_distance": 1e9,
    28	        "stage3_gather_16bit_weights

In [4]:
import torch


torch.cuda.is_available()

True

In [11]:
import torch
import deepspeed
from transformers.deepspeed import HfDeepSpeedConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import json
import os


# model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
model_name = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"


# multi-GPU関連の設定
os.environ[
    "TOKENIZERS_PARALLELISM"
] = "false"  # To avoid warnings about parallelism in tokenizers
local_rank = int(os.getenv("LOCAL_RANK", 0))
world_size = int(os.getenv("WORLD_SIZE", 1))

torch.cuda.set_device(local_rank)
deepspeed.init_distributed()

# ベースとなるZeRO3 configの読み込み
ds_config_file = "zero_infer.json"
with open(ds_config_file) as f:
    ds_config = json.load(f)

# 推論用に修正
model_config = AutoConfig.from_pretrained(model_name)
hidden_size = model_config.hidden_size

ds_config["train_batch_size"] = 1 * world_size
ds_config["train_micro_batch_size_per_gpu"] = 1
ds_config["reduce_bucket_size"] = hidden_size * hidden_size
ds_config["stage3_prefetch_bucket_size"] = 0.9 * hidden_size * hidden_size
ds_config["stage3_param_persistence_threshold"] = 10 * hidden_size


dschf = HfDeepSpeedConfig(ds_config)  # zero3を使用するために必要(モデルロード前に実行する必要がある)

In [12]:
# load LLM
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2024-01-03 20:27:56,512] [INFO] [partition_parameters.py:347:__exit__] finished initializing model - num_params = 582, num_elems = 13.69B


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# setup deepspeed
ds_engine = deepspeed.initialize(model=model, config_params=ds_config)[0]
ds_model = ds_engine.module.eval()

[2024-01-03 19:08:31,074] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.10.3, git-hash=unknown, git-branch=unknown
[2024-01-03 19:08:31,094] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-01-03 19:08:31,097] [INFO] [logging.py:96:log_dist] [Rank 0] Creating ZeRO Offload
[2024-01-03 19:08:31,184] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2024-01-03 19:08:31,185] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 1.16 GB         CA 1.44 GB         Max_CA 1 GB 
[2024-01-03 19:08:31,186] [INFO] [utils.py:811:see_memory_usage] CPU Virtual Memory:  used = 23.45 GB, percent = 37.4%
Parameter Offload: Total persistent parameters: 266240 in 65 params
[2024-01-03 19:08:31,286] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2024-01-03 19:08:31,287] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 0.06 GB         CA 1.44 GB         Max_CA 1

In [8]:
from transformers import pipeline


if torch.cuda.is_available():
  model.to("cuda")


def gen(input_text="今日の天気は"):
    text_pipe = pipeline('text-generation', 
                         model=ds_model,
                         tokenizer=tokenizer,
                         device="cuda:0",
                         max_length=200,
                             )
    output = text_pipe(input_text)

    return output[0]['generated_text']
%time gen()


[2024-01-03 19:08:39,150] [WARNING] [parameter_offload.py:86:_apply_to_tensors_only] A module has unknown inputs or outputs type (<class 'transformers.cache_utils.DynamicCache'>) and the tensors embedded in it cannot be detected. The ZeRO-3 hooks designed to trigger before or after backward pass of the module relies on knowing the input and output tensors and therefore may not get triggered properly.
CPU times: user 7min 2s, sys: 1.57 s, total: 7min 3s
Wall time: 7min 8s


'今日の天気は、晴れ。\n最高気温は、22℃。\n風は、北風。\n今日は、1日、休日です。\n月曜日なので、出勤の人が多いですが、休みの人も多いので、混雑は、少ないでしょう。\n今日は、仕事が、終わったので、家事を済ませ、15時頃、外出しました。\n16時頃、外出したので、まだ、日が、明るいです。\n17時頃、外出したので、もう、暗くなっています。\n18時頃、外出したので、もう、暗くて、怖いです。\n19時頃、外出したので、もう、暗くて、怖いです。\n20時頃、外出したので、もう、暗くて、怖いです。\n21時頃、外出したので、もう、暗くて、怖いです。\n'

In [9]:
# model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")
#
# if torch.cuda.is_available():
#     model = model.to("cuda:1")
#
# prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
#     bos_token=tokenizer.bos_token,
#     b_inst=B_INST,
#     system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
#     prompt=text,
#     e_inst=E_INST,
# )

In [10]:
# with torch.no_grad():
#     token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
#
#     output_ids = model.generate(
#         token_ids.to(model.device),
#         max_new_tokens=256,
#         pad_token_id=tokenizer.pad_token_id,
#         eos_token_id=tokenizer.eos_token_id,
#     )
# output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
# print(output)
# """
# 承知しました。以下にクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を記述します。
#
# クマは山の中でゆっくりと眠っていた。
# その眠りに落ちたクマは、夢の中で海辺を歩いていた。
# そこにはアザラシがいた。
# クマはアザラシに話しかける。
#
# 「おはよう」とクマが言うと、アザラシは驚いたように顔を上げた。
# 「あ、こんにちは」アザラシは答えた。
# クマはアザラシと友達になりたいと思う。
#
# 「私はクマと申します。」クマは...
# """
#